<a href="https://colab.research.google.com/github/SirSirocco/DataScience_2025_1/blob/main/experiment_plan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DEPENDÊNCIAS GERAIS

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import joblib
import warnings

from google.colab import drive
from itertools import product
from sklearn.base import BaseEstimator, clone
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import KFold, train_test_split
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor

# FUNÇÕES

## Utilitário

In [2]:
def dict_combine(dictionary: dict) -> list:
    """
    Gera todas as combinações possíveis entre os valores de um dicionário de listas,
    retornando uma lista de dicionários com cada combinação única.

    :param dictionary: Dicionário em que cada chave está associada a uma lista de valores possíveis.
                       Os valores devem ser iteráveis (como listas ou tuplas) e de mesmo comprimento.

    :return: Lista contendo dicionários. Cada dicionário representa uma combinação possível
             entre os valores das listas fornecidas no dicionário original. As chaves são mantidas,
             e os valores correspondem a uma das combinações do produto cartesiano.
    """
    # Passo 1: Obtemos as chaves do dicionário (ex: ["key1", "key2"])
    keys = list(dictionary.keys())

    # Passo 2: Obtemos os valores associados a cada chave (listas de possibilidades)
    # Exemplo: [["a", "b", "c"], ["d", "e", "f"]]
    values = list(dictionary.values())

    # Passo 3: Calculamos o produto cartesiano dessas listas de valores
    # Isso gera todas as combinações possíveis, como ("a", "d"), ("a", "e"), ..., ("c", "f")
    combinations = product(*values)  # Asterisco faz o desempacotamento dos elementos de values

    # Passo 4: Para cada combinação, associamos os valores às suas respectivas chaves
    # Isso é feito com a função zip, e transformamos o resultado em um dicionário
    # Resultado final: uma lista de dicionários, cada um representando uma combinação possível
    return [dict(zip(keys, combo)) for combo in combinations]

def dict_flatten(dictionary: dict, fields_to_flatten: list[str]) -> dict:
    """
    Desestrutura os campos selecionados de um dicionário, retornando apenas os atributos desejados.

    :param dictionary: Dicionário original que contém os campos a serem desestruturados.
    :param fields_to_flatten: Lista de nomes dos campos que devem ser desestruturados (flatten).

    :return: Novo dicionário contendo apenas os atributos dos campos selecionados, combinados em um nível.
    """
    dict_flat = dict()

    for field in fields_to_flatten:
        value = dictionary.get(field)
        if isinstance(value, dict):
            dict_flat.update(value)

    return dict_flat

def dict_to_flat_df(dictionary: dict, fields_to_flatten: list[str], key_name: str = "key") -> pd.DataFrame:
    """
    Converte um dicionário com estrutura aninhada em um DataFrame tabular, desestruturando
    apenas os campos especificados.

    :param dictionary: Dicionário em que cada chave representa um identificador único e cada valor é um dicionário com possíveis campos aninhados.
    :param fields_to_flatten: Lista de nomes de campos a serem desestruturados. Cada um deve
                               corresponder a uma chave cujo valor é um dicionário.
    :param key_name: Nome da coluna que armazenará os identificadores do dicionário original.
                     Padrão é "key".

    :return: DataFrame com uma coluna para os identificadores e colunas adicionais contendo os
             atributos resultantes da desestruturação dos campos selecionados.
    """
    rows = list()

    for identifier, nested_dict in dictionary.items():
        dict_flat = dict_flatten(nested_dict, fields_to_flatten)
        dict_flat[key_name] = identifier
        rows.append(dict_flat)

    df = pd.DataFrame(rows)
    df = df[[key_name] + [col for col in df.columns if col != key_name]]  # Coloca a chave primeiro
    return df

def key_build(key_parts: list[str], separator: str = "_") -> str:
    return separator.join(key_parts)

def key_build_from_params(params: dict[str]):
    parts = [key_build([str(k).upper(), str(v).lower()]) for k, v in params.items()]
    return key_build(parts)

## Exibição

In [3]:
def df_show_head(df: pd.DataFrame, n: int = 5) -> None:
    display(df.head(n))
    print(f"Shape: {df.shape}")

## Validação

In [4]:
def is_valid_col_selection(col_selection: dict) -> bool:
    return col_selection is not None

def is_valid_series(se: pd.Series) -> bool:
    return se is not None and (not se.empty) and se.notnull().all() and pd.api.types.is_numeric_dtype(se)

## Pré-Processamento

In [5]:
def subs_outliers(df: pd.DataFrame, columns: list[str], view: bool = False) -> pd.DataFrame:
    """
    Substitui outliers pela média. Se view for True, exibe
    dois gráficos para cada coluna: antes e depois da mudança.
    """

    # Guarda resultado anterior
    df_before = df.copy()

    for column in columns:
        # Obtém quartis 1 e 3
        q1 = df[column].quantile(0.25)
        q3 = df[column].quantile(0.75)

        # Obtém IQR (Intervalo Interquartílico)
        iqr = q3 - q1

        """
        Outlier se em (-inf, Q1 - 1.5 * IQR) ou (Q3 + 1.5 * IQR, +inf).
        Subsititui outliers por NA ('not available') segundo método IQR (InterQuartile Range).
        """
        df.loc[(df[column] < q1 - 1.5 * iqr) | (df[column] > q3 + 1.5 * iqr), column] = pd.NA

        # Substitui NAs pela média
        df[column] = df[column].fillna(df[column].mean())

    if view:
        for column in columns:
            # Exibe resultado inicial
            plt.boxplot(df_before[column])
            plt.title(f"ANTES: {column}")
            plt.show()

            # Exibe novo resultado
            plt.boxplot(df[column])
            plt.title(f"DEPOIS: {column}")
            plt.show()

    return df

## Validação Cruzada

In [6]:
def cross_val(model: BaseEstimator,
              X: pd.DataFrame,
              y: pd.Series,
              folds: int = 5,
              metric: str = "r2",
              shuffle: bool = True,
              random_state: int = None) -> tuple[list, list]:
    """
    Realiza validação cruzada utilizando K-Fold para treinar e avaliar um modelo,
    com base em uma métrica de desempenho fornecida.

    O modelo é clonado para cada divisão dos dados, garantindo independência entre os folds.

    :param model: Estimador compatível com o Scikit-learn (deve implementar os métodos `fit` e `predict`).
    :param X: Conjunto de dados de entrada (features).
    :param y: Vetor alvo (target) correspondente às amostras em `X`.
    :param folds: Número de partições (folds) na validação cruzada. O padrão é 5.
    :param metric: Métrica de avaliação utilizada em cada fold.
                   Pode ser uma das opções: `"r2"`, `"mse"`, `"mae"`.
                   Se inválida, `"r2"` será usada como padrão.
    :param shuffle: Indica se os dados devem ser embaralhados antes da divisão em folds.
    :param random_state: Semente de aleatoriedade para reprodutibilidade. `None` implica aleatoriedade completa.

    :return: Tupla contendo:
             - lista de modelos treinados (um por fold)
             - lista de métricas correspondentes a cada fold
    """
    DICT_METRICS = {
        "r2":   r2_score,
        "mse":  mean_squared_error,
        "mae":  mean_absolute_error
    }
    models = list() # Salva modelos por fold
    metrics = list() # Salva resultados por fold
    metric_to_use = DICT_METRICS.get(metric, r2_score) # Define métrica a ser usada (padrão é coeficiente de determinação)

    # Define os índices que delimitam as partições
    kfolds = KFold(n_splits=folds, shuffle=shuffle, random_state=random_state)

    for train_index, test_index in kfolds.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        copy = clone(model) # Evita sobrescrição do modelo
        copy.fit(X_train, y_train)

        models.append(copy)
        metrics.append(metric_to_use(y_test, copy.predict(X_test))) # Calcula métrica

    return models, metrics

## Casos de pré-processamento

In [24]:
# --- CONSTANTES SIMBÓLICAS ---
"""
Incluímos '_' ao início para evitar conflitos de nomes com o restante do código.
"""
# - Pré-processamento -
_COLUMN_REMOVAL = "col_rem"
_COLUMNS = "columns"
_NAME = "name"
_FEATURE_ENGINEERING = "feat_eng"
_OUTLIERS = "outliers"
_NORMALIZATION = "normalization"
_PCA = "pca"

PREFIXES = {
    _COLUMN_REMOVAL: "FS",
    _FEATURE_ENGINEERING: "FE",
    _OUTLIERS: "OUT",
    _NORMALIZATION: "NORM",
    _PCA: "PCA"
}

_ABSENT = "absent"
_TRUE = "true"
_FALSE = "false"

RADIXES = {
    _ABSENT: "na",
    _TRUE: "1",
    _FALSE: "0"
}

_X = "X"
_y = "y"
_PARAMS = "params"

# - Métricas -
_MAE = "mae"
_MSE = "mse"
_R2 = "r2"
DICT_METRICS = {
        _MAE: mean_absolute_error,
        _MSE: mean_squared_error,
        _R2:  r2_score,
}
_AVERAGE = "avg"
_BEST_MODEL = "best_model"
_CROSS_VAL = "cv"
_FOLD = "fold"
_TEST = "test"
_PRED = "y_pred"

# FUNÇÕES
def apply_column_removal(df, X, y, col_selection, prefix, radixes):
    key = prefix

    if is_valid_col_selection(col_selection):
        columns = col_selection[_COLUMNS]
        df = df.drop(columns=columns)
        key = key_build([key, col_selection[_NAME].lower()]) # Indica nome da seleção de colunas
    else:
        key = key_build([key, radixes[_ABSENT]])
        columns = None

    return df, X, y, key, {_COLUMN_REMOVAL: columns}

def apply_feature_engineering(df, X, y, feat_eng, prefix, radixes):
    key = prefix

    if is_valid_series(feat_eng):
        df = pd.concat([df, feat_eng], axis=1)
        key = key_build([key, feat_eng.name.lower()]) # Indica nome da característica
        feat_name = feat_eng.name
    else:
        key = key_build([key, radixes[_ABSENT]])
        feat_name = None

    return df, X, y, key, {_FEATURE_ENGINEERING: feat_name}

def apply_outliers(df, X, y, out, prefix, radixes):
    key = prefix

    if out:
        df = subs_outliers(df, df.columns, view=False)
        key += radixes[_TRUE]
    else:
        key += radixes[_FALSE]

    X = df.drop(columns=[y.name])
    y = df[y.name].squeeze() # Squeeze garante ser uma série

    return df, X, y, key, {_OUTLIERS: out}

apply_normalization_scaler = StandardScaler() # Guarda scaler para reuso
def apply_normalization(df, X, y, norm, prefix, radixes):
    key = prefix

    if norm:
        X = pd.DataFrame(apply_normalization_scaler.fit_transform(X), columns=X.columns, index=X.index)
        key += radixes[_TRUE]
    else:
        key += radixes[_FALSE]

    return df, X, y, key, {_NORMALIZATION: norm}

def apply_pca(df, X, y, pca_n, prefix, radixes):
    key = f"{prefix}{pca_n}"

    # Evita que componentes superem total de colunas, o que geraria redundância
    pca_n = min(len(X.columns), pca_n)

    pca_obj = PCA(n_components=pca_n)
    pca_arr = pca_obj.fit_transform(X)
    X_pca = pd.DataFrame(
        pca_arr,
        index=X.index,
        columns=[f"pc{i + 1}" for i in range(pca_n)]
    )

    return df, X_pca, y, key, {_PCA: pca_n}

def get_preprocessing_cases(df: pd.DataFrame,
                            target: str,
                            random_state: int,
                            col_selections: list[dict],
                            features: list[pd.Series],
                            outliers: list[bool],
                            normalization: list[bool],
                            pcas: list[int]) -> dict:
    """
    Gera múltiplos cenários de pré-processamento combinando, seleção e engenharia de atributos,
    tratamento de outliers, normalização e redução de dimensionalidade (PCA).

    Cada cenário gera um conjunto `X`, `y` e um dicionário com parâmetros das transformações aplicadas.
    As chaves do dicionário final descrevem o pipeline aplicado via prefixos e sufixos codificados.

    :param df: DataFrame original contendo os dados brutos.
    :param target: Nome da coluna alvo (variável dependente) no DataFrame.
    :param random_state: Semente para controle de aleatoriedade (reprodutibilidade).
    :param col_selection: Lista de dicionários com atributos "name" (representação do cenário) e "columns" (lista de colunas a serem removidas).
    :param features: Lista de séries representando atributos derivados para engenharia de características.
                     Pode conter séries válidas ou `None` para ausência de feature engineering.
    :param outliers: Lista indicando se o tratamento de outliers deve ser aplicado.
    :param normalization: Lista indicando se a normalização (z-score) deve ser aplicada.
    :param pcas: Lista com números de componentes principais a serem testados via PCA.

    :return: Dicionário cujas chaves são strings descritivas do pipeline (ex: 'FEna_OUT1_NORM0_PCA2'), e
             os valores são dicionários contendo:
             - `_X`: DataFrame com os atributos processados.
             - `_y`: Série da variável alvo.
             - `_PARAMS`: Parâmetros que descrevem as transformações aplicadas no cenário.
    """
    dict_cases = dict() # Armazena os casos de experimentação
    df_original = df.copy() # Salva estado original do DataFrame

    np.random.seed(random_state) # Fixa semente aleatória

    for select, feat, out, norm, pca in product(col_selections, features, outliers, normalization, pcas):
        # Etapas de pré-processamento
        steps = [
            (apply_column_removal, select, PREFIXES[_COLUMN_REMOVAL]),
            (apply_feature_engineering, feat, PREFIXES[_FEATURE_ENGINEERING]),
            (apply_outliers, out, PREFIXES[_OUTLIERS]),
            (apply_normalization, norm, PREFIXES[_NORMALIZATION]),
            (apply_pca, pca, PREFIXES[_PCA]),
        ]

        key_parts = list()          # Armazena partes da chave que descreve o caso
        dict_params = dict()        # Armazena os parâmetros que compõem o caso
        df = df_original.copy()     # Copia DataFrame original para evitar sobrescrição
        X = df.drop(columns=[target])
        y = df[target]

        # Executa etapas
        for function, param, prefix in steps:
            df, X, y, key, update = function(df, X, y, param, prefix, RADIXES)
            key_parts.append(key)
            dict_params.update(update)

        key = key_build(key_parts)
        dict_cases[key] = {_X: X, _y: y, _PARAMS: dict_params}

    return dict_cases

def get_best_model(metric, models, scores):
    DICT_CRITERIA = {
        _MAE: np.argmin,
        _MSE: np.argmin,
        _R2: np.argmax
    }
    best_index = DICT_CRITERIA[metric](scores)
    best_index = best_index[0] if isinstance(best_index, np.ndarray) else best_index
    return models[best_index]

def get_model_cases(model: BaseEstimator,
                    pre_processing_cases: dict,
                    params: dict,
                    folds: list[int],
                    metrics: list[str],
                    test_size: float,
                    random_state: int) -> dict:
    """
    Executa experimentos completos de treinamento e avaliação de um modelo para múltiplos
    cenários de pré-processamento e combinações de hiperparâmetros.

    Para cada combinação de cenário de dados, configuração de hiperparâmetros e número de folds,
    realiza validação cruzada, seleciona o melhor modelo segundo uma métrica especificada e avalia
    seu desempenho em um conjunto de teste.

    :param model: Estimador compatível com Scikit-learn que será ajustado aos dados.
    :param pre_processing_cases: Dicionário de cenários de pré-processamento, em que cada chave representa
                                 um pipeline aplicado e os valores contêm os dados transformados (`X`, `y`).
    :param params: Dicionário de listas de valores a serem combinados como hiperparâmetros do modelo.
    :param folds: Lista com os diferentes valores de `k` a serem usados na validação cruzada (K-Fold).
    :param metrics: Lista de métricas a serem utilizadas para avaliação (`"r2"`, `"mae"`, `"mse"`).
    :param test_size: Proporção dos dados reservada para teste na separação treino-teste.
    :param random_state: Semente para garantir reprodutibilidade na separação dos dados e na validação cruzada.

    :return: Dicionário em que cada chave representa um cenário único (pipeline + hiperparâmetros) e
             os valores contêm:
                - `_PARAMS`: dicionário de hiperparâmetros usados no modelo
                - `_BEST_MODEL`: dicionário com os melhores modelos por métrica (escolhidos com base na V.C.)
                - `_CROSS_VAL`: dicionário com as métricas por fold e médias das validações cruzadas
                - `_TEST`: dicionário com as métricas obtidas no conjunto de teste por modelo selecionado
                - `_PRED`: dicionário com as predições (`y_pred`) dos melhores modelos no conjunto de teste,
                permitindo reconstruir resíduos
    """
    dict_cases = dict() # Armazena os casos de experimentação
    model_name = model.__class__.__name__

    # Percorre cenários de pré-processamento
    for base_key, pre_proc_params in pre_processing_cases.items():
        # Inicio da nova chave formado pela chave do pré-processamento e nome do modelo
        base_key = key_build([base_key, model_name])

        # Gera conjuntos de treino e de teste
        X = pre_proc_params[_X]
        y = pre_proc_params[_y]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

        # Gera cenários com base no número de folds e nos parâmetros do modelo
        for model_params, fold_num in product(dict_combine(params), folds):
            dict_best_models = dict() # Armazena melhores modelos por métrica
            dict_scores_cross_val = dict() # Armazena pontuações médias da V.C. por métrica
            dict_scores_test = dict() # Armazena pontuações do teste por métrica
            dict_y_pred_test = dict() # Armazena predições do teste por métrica

            model_case = clone(model)
            model_case.set_params(**model_params)

            for metric in metrics:
                models_cross_val, scores_cross_val = cross_val(model_case,
                                    X_train,
                                    y_train,
                                    folds=fold_num,
                                    metric=metric,
                                    shuffle=True,
                                    random_state=random_state)

                # Guarda métricas por fold
                for fold, score in zip(range(fold_num), scores_cross_val):
                    key_cross_val_folds = key_build([metric, _CROSS_VAL, _FOLD, str(fold)])
                    dict_scores_cross_val[key_cross_val_folds] = score

                # Computa métrica média
                key_cross_val_avg = key_build([metric, _CROSS_VAL, _AVERAGE])
                dict_scores_cross_val[key_cross_val_avg] = np.mean(scores_cross_val)

                # Escolhe melhor modelo
                key_cross_val_best_model = key_build([metric, _CROSS_VAL, _BEST_MODEL])
                best_model = get_best_model(metric, models_cross_val, scores_cross_val)
                dict_best_models[key_cross_val_best_model] = best_model

                # Computa métricas do melhor modelo
                y_pred = best_model.predict(X_test)
                for internal_metric in metrics:
                    key_best_model_test = key_build([metric, _BEST_MODEL, _TEST, internal_metric])
                    dict_scores_test[key_best_model_test] = DICT_METRICS[internal_metric](y_test, y_pred)

                # Armazena predições para reconstrução futura dos resíduos
                key_y_pred_test = key_build([metric, _BEST_MODEL, _TEST, _PRED])
                dict_y_pred_test[key_y_pred_test] = y_pred

            # Gera nova chave
            key_model_params = key_build_from_params(model_params)
            key_new = key_build([base_key, key_model_params])

            dict_cases[key_new] = {
                _PARAMS: {**model_params},
                _BEST_MODEL: dict_best_models,
                _CROSS_VAL: dict_scores_cross_val,
                _TEST: dict_scores_test,
                _PRED: dict_y_pred_test
            }

    return dict_cases

# SALVAMENTO

In [8]:
def dict_deserialize(path: str) -> dict:
    """
    Desserializa (carrega) um dicionário Python a partir de um arquivo .pkl.

    :param path: Caminho completo do arquivo a ser carregado.
                 Deve terminar com a extensão '.pkl' (ex: 'dict_cases.pkl').
    :return: Dicionário Python carregado a partir do arquivo.
    """
    return joblib.load(path)

def dict_serialize(dictionary: dict, path: str) -> None:
    """
    Serializa (salva) um dicionário Python em um arquivo no formato .pkl.

    :param dictionary: Dicionário a ser serializado.
    :param path: Caminho completo do arquivo de destino.
                 Deve terminar com a extensão '.pkl' (ex: 'dict_cases.pkl').
    """
    joblib.dump(dictionary, path)

def generate_sheet(dict_cases: dict, sheet_name: str, path: str) -> None:
    """
    Gera uma planilha Excel (.xlsx) contendo os dados tabulares dos casos modelados.

    :param dict_cases: Dicionário onde cada chave identifica um caso e o valor é um
                       dicionário com os dados do modelo, parâmetros e métricas.
    :param sheet_name: Nome da aba (sheet) que será criada no arquivo Excel.
    :param path: Caminho completo do arquivo .xlsx que será criado.
                 Deve terminar com a extensão '.xlsx' (ex: 'resultados.xlsx').

    A função realiza o seguinte:
    - Achata os campos definidos em FLATTEN (campos dicionário), elevando-os ao primeiro nível.
    - Concatena os dados em um único DataFrame.
    - Escreve esse DataFrame em uma planilha Excel na aba especificada.
    """
    FLATTEN = [_PARAMS, _CROSS_VAL, _TEST] # Campos a serem passados para o primeiro nível
    KEY_NAME = "case" # Nome do campo que armazenará o identificador dos casos

    """
    Achata campos que são dicionários, deixando todas as informações no primeiro
    nível. Em seguida, transforma em um DataFrame.
    """
    df_final = dict_to_flat_df(dict_cases, FLATTEN, KEY_NAME)

    with pd.ExcelWriter(path, engine="openpyxl") as writer:
        df_final.to_excel(writer, sheet_name=sheet_name, index=False)

# SETUP DO AMBIENTE

In [9]:
drive.mount("/content/drive")

Mounted at /content/drive


# OBTENÇÃO DO DATASET

In [10]:
PATH = "/content/drive/MyDrive/07_per_shared/projCDat_25_1/datasets/cooked/_all/all_merged.csv"
df = pd.read_csv(PATH)
df_show_head(df)

,_ano,_estado,_mes,car_c02_emitido,cli_pressao_atm_med,cli_temp_ar_med,cli_temp_orvalho_med,cli_umid_rel_med,cli_umid_rel_min_max,cli_umid_rel_min_med,cli_umid_rel_min_min,cli_veloc_vento_max,cli_veloc_vento_med,que_area_queimada,que_focos_qtd
0,2008,AC,7,2.627698e+07,986.843612,28.142731,18.914978,59.555066,95.0,54.432558,29.0,5.1,2.152915,4957.0,165.0
1,2008,AC,9,2.627698e+07,991.705941,24.446194,19.467987,75.811881,97.0,72.496700,25.0,1.0,0.210504,46073.0,2947.0
2,2008,AC,10,2.627698e+07,990.328360,25.229298,21.617473,81.870968,96.0,78.720430,29.0,1.0,0.204959,30355.0,856.0
3,2008,AC,11,2.627698e+07,988.610987,25.195410,22.624478,86.905424,96.0,84.048679,42.0,1.0,0.186970,2082.0,63.0
4,2008,AC,12,2.627698e+07,988.692608,24.898790,22.727554,88.529570,96.0,86.116935,53.0,1.0,0.179442,127.0,4.0


Shape: (1025, 15)


# CODIFICAÇÃO

In [11]:
# Codificação OneHot simplificada
TO_ENCODE = ["_estado"]
df_encoded = pd.get_dummies(df, columns=TO_ENCODE, dtype="Int32")

df_show_head(df_encoded)

,_ano,_mes,car_c02_emitido,cli_pressao_atm_med,cli_temp_ar_med,cli_temp_orvalho_med,cli_umid_rel_med,cli_umid_rel_min_max,cli_umid_rel_min_med,cli_umid_rel_min_min,...,cli_veloc_vento_med,que_area_queimada,que_focos_qtd,_estado_AC,_estado_AM,_estado_AP,_estado_PA,_estado_RO,_estado_RR,_estado_TO
0,2008,7,2.627698e+07,986.843612,28.142731,18.914978,59.555066,95.0,54.432558,29.0,...,2.152915,4957.0,165.0,1,0,0,0,0,0,0
1,2008,9,2.627698e+07,991.705941,24.446194,19.467987,75.811881,97.0,72.496700,25.0,...,0.210504,46073.0,2947.0,1,0,0,0,0,0,0
2,2008,10,2.627698e+07,990.328360,25.229298,21.617473,81.870968,96.0,78.720430,29.0,...,0.204959,30355.0,856.0,1,0,0,0,0,0,0
3,2008,11,2.627698e+07,988.610987,25.195410,22.624478,86.905424,96.0,84.048679,42.0,...,0.186970,2082.0,63.0,1,0,0,0,0,0,0
4,2008,12,2.627698e+07,988.692608,24.898790,22.727554,88.529570,96.0,86.116935,53.0,...,0.179442,127.0,4.0,1,0,0,0,0,0,0


Shape: (1025, 21)


# ENGENHARIA DE CARACTERÍSTICAS

In [12]:
se_spreading = df_encoded["cli_temp_ar_med"] * df_encoded["cli_veloc_vento_med"] * df_encoded["que_focos_qtd"]
se_spreading.name = "fire_spreading"
display(se_spreading)

,fire_spreading
0,9997.168934
1,15165.340107
2,4426.344890
3,296.779783
4,17.871555
...,...
1020,62002.830769
1021,140420.048771
1022,70471.339648
1023,6029.214615


# SELEÇÃO DE CARACTERÍSTICAS

Abaixo, consideram-se dois cenários de _feature selection_.

a remoção de variáveis que possuam elevado coeficiente de correlação de Pearson

In [13]:
# Remoção de variáveis com coeficiente de correlação (em módulo) elevado
scn_not_corr = {
    "name": "not_corr",
    "columns": [
        "cli_temp_orvalho_med",
        "cli_umid_rel_med",
        "cli_umid_rel_min_min",
        "cli_veloc_vento_max"
    ]
}

# Remoção dos estados
scn_stateless = {
    "name": "stateless",
    "columns": [col for col in df_encoded.columns if col.startswith("_estado")]
}

# SETUP

In [14]:
# Constantes simbólicas
PRE = "pre_processing"     # Pré-processamento
RF = "random_forest"  # Random Forest
XGB = "xgboost" # XGBoost
MLR = "multiple_linear_regression" # Regressão Linear Múltipla
ALL = "all"     # Todos os modelos considerados simultaneamente

# Dicionários para armazenar os resultados
dict_cases = dict() # Armazena dicionários de casos
dict_df_cases = dict() # Armazena DataFrames de casos

# PARÂMETROS GERAIS
RANDOM_STATE = 42
TARGET = "que_area_queimada"
CONFIG = {
    "folds": [3],
    "metrics": [_MAE, _MSE, _R2],
    "test_size": 0.30,
    "random_state": RANDOM_STATE
}

# SALVAMENTO
PATH_OUT = "/content/drive/MyDrive/07_per_shared/projCDat_25_1/src_sprint03/plans"

# CASOS

## CASOS DE PRÉ-PROCESSAMENTO

In [15]:
# Parâmetros auxiliares (mude se desejar)
PRE_PARAMS = {
    "df": df_encoded,
    "target": TARGET,
    "random_state": RANDOM_STATE,
    "col_selections": [None, scn_not_corr , scn_stateless],
    "features": [None, se_spreading],
    "outliers": [False, True],
    "normalization": [False, True],
    "pcas": [x for x in range(5, 21, 5)]
}

# Obtém casos de pré-processamento
dict_cases[PRE] = get_preprocessing_cases(**PRE_PARAMS)
dict_df_cases[PRE] = pd.DataFrame(data=dict_cases[PRE]).T

df_show_head(dict_df_cases[PRE])
dict_serialize(dict_cases[PRE], f"{PATH_OUT}/{PRE}.pkl")
generate_sheet(dict_cases[PRE], PRE, f"{PATH_OUT}/{PRE}.xlsx")

,X,y,params
FS_na_FE_na_OUT0_NORM0_PCA5,pc1 pc2 pc3 ...,0 4957.0 1 46073.0 2 303...,"{'col_rem': None, 'feat_eng': None, 'outliers'..."
FS_na_FE_na_OUT0_NORM0_PCA10,pc1 pc2 pc3 ...,0 4957.0 1 46073.0 2 303...,"{'col_rem': None, 'feat_eng': None, 'outliers'..."
FS_na_FE_na_OUT0_NORM0_PCA15,pc1 pc2 pc3 ...,0 4957.0 1 46073.0 2 303...,"{'col_rem': None, 'feat_eng': None, 'outliers'..."
FS_na_FE_na_OUT0_NORM0_PCA20,pc1 pc2 pc3 ...,0 4957.0 1 46073.0 2 303...,"{'col_rem': None, 'feat_eng': None, 'outliers'..."
FS_na_FE_na_OUT0_NORM1_PCA5,pc1 pc2 pc3 pc4 ...,0 4957.0 1 46073.0 2 303...,"{'col_rem': None, 'feat_eng': None, 'outliers'..."


Shape: (96, 3)


## Regressão Linear Múltipla

In [25]:
# Parâmetros auxiliares (mude se desejar)
MLR_PARAMS = {
    "alpha": [1e-4, 1e-3, 1e-2, 0.1, 1.0, 10.0],
    "fit_intercept": [True, False],
    "max_iter": [1000],
    "random_state": [RANDOM_STATE]
}

dict_cases[MLR] = get_model_cases(
    model=Lasso(),
    params=MLR_PARAMS,
    pre_processing_cases=dict_cases[PRE],
    **CONFIG
)
dict_df_cases[MLR] = pd.DataFrame(data=dict_cases[MLR]).T

df_show_head(dict_df_cases[MLR])
dict_serialize(dict_cases[MLR], f"{PATH_OUT}/{MLR}.pkl")
generate_sheet(dict_cases[MLR], MLR, f"{PATH_OUT}/{MLR}.xlsx")

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.561e+12, tolerance: 3.741e+09
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.561e+12, tolerance: 3.741e+09
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.561e+12, tolerance: 3.741e

                                                                                               params  \
FS_na_FE_na_OUT0_NORM0_PCA5_Lasso_ALPHA_0.0001_...  {'alpha': 0.0001, 'fit_intercept': True, 'max_...   
FS_na_FE_na_OUT0_NORM0_PCA5_Lasso_ALPHA_0.0001_...  {'alpha': 0.0001, 'fit_intercept': False, 'max...   
FS_na_FE_na_OUT0_NORM0_PCA5_Lasso_ALPHA_0.001_F...  {'alpha': 0.001, 'fit_intercept': True, 'max_i...   
FS_na_FE_na_OUT0_NORM0_PCA5_Lasso_ALPHA_0.001_F...  {'alpha': 0.001, 'fit_intercept': False, 'max_...   
FS_na_FE_na_OUT0_NORM0_PCA5_Lasso_ALPHA_0.01_FI...  {'alpha': 0.01, 'fit_intercept': True, 'max_it...   
...                                                                                               ...   
FS_stateless_FE_fire_spreading_OUT1_NORM1_PCA20...  {'alpha': 0.1, 'fit_intercept': False, 'max_it...   
FS_stateless_FE_fire_spreading_OUT1_NORM1_PCA20...  {'alpha': 1.0, 'fit_intercept': True, 'max_ite...   
FS_stateless_FE_fire_spreading_OUT1_NORM1_PCA20...  {'a

,params,best_model,cv,test,y_pred
FS_na_FE_na_OUT0_NORM0_PCA5_Lasso_ALPHA_0.0001_FIT_INTERCEPT_true_MAX_ITER_1000_RANDOM_STATE_42,"{'alpha': 0.0001, 'fit_intercept': True, 'max_...","{'mae_cv_best_model': Lasso(alpha=0.0001, rand...","{'mae_cv_fold_0': 105139.23445429234, 'mae_cv_...","{'mae_best_model_test_mae': 103522.6826492222,...",{'mae_best_model_test_y_pred': [416105.2381557...
FS_na_FE_na_OUT0_NORM0_PCA5_Lasso_ALPHA_0.0001_FIT_INTERCEPT_false_MAX_ITER_1000_RANDOM_STATE_42,"{'alpha': 0.0001, 'fit_intercept': False, 'max...","{'mae_cv_best_model': Lasso(alpha=0.0001, fit_...","{'mae_cv_fold_0': 133247.70929302793, 'mae_cv_...",{'mae_best_model_test_mae': 139865.59926129726...,{'mae_best_model_test_y_pred': [345750.9181885...
FS_na_FE_na_OUT0_NORM0_PCA5_Lasso_ALPHA_0.001_FIT_INTERCEPT_true_MAX_ITER_1000_RANDOM_STATE_42,"{'alpha': 0.001, 'fit_intercept': True, 'max_i...","{'mae_cv_best_model': Lasso(alpha=0.001, rando...","{'mae_cv_fold_0': 105139.23443836758, 'mae_cv_...",{'mae_best_model_test_mae': 103522.68263235355...,{'mae_best_model_test_y_pred': [416105.2381360...
FS_na_FE_na_OUT0_NORM0_PCA5_Lasso_ALPHA_0.001_FIT_INTERCEPT_false_MAX_ITER_1000_RANDOM_STATE_42,"{'alpha': 0.001, 'fit_intercept': False, 'max_...","{'mae_cv_best_model': Lasso(alpha=0.001, fit_i...","{'mae_cv_fold_0': 133247.70926758007, 'mae_cv_...",{'mae_best_model_test_mae': 139865.59922585366...,{'mae_best_model_test_y_pred': [345750.9181839...
FS_na_FE_na_OUT0_NORM0_PCA5_Lasso_ALPHA_0.01_FIT_INTERCEPT_true_MAX_ITER_1000_RANDOM_STATE_42,"{'alpha': 0.01, 'fit_intercept': True, 'max_it...","{'mae_cv_best_model': Lasso(alpha=0.01, random...","{'mae_cv_fold_0': 105139.23427912022, 'mae_cv_...",{'mae_best_model_test_mae': 103522.68246366676...,{'mae_best_model_test_y_pred': [416105.2379393...


Shape: (1152, 5)


## Random Forest

In [ ]:
# Parâmetros auxiliares (mude se desejar)
RF_PARAMS = {
    "n_estimators": [100, 200, 400],
    "criterion": ["friedman_mse", "squared_error"],
    "max_depth": [2, 6, 10],
    "random_state": [RANDOM_STATE]
}

dict_cases[RF] = get_model_cases(
    model=RandomForestRegressor(),
    params=RF_PARAMS,
    pre_processing_cases=dict_cases[PRE],
    **CONFIG
)
dict_df_cases[RF] = pd.DataFrame(data=dict_cases[RF]).T

df_show_head(dict_df_cases[RF])
dict_serialize(dict_cases[RF], f"{PATH_OUT}/{RF}.pkl")
generate_sheet(dict_cases[RF], RF, f"{PATH_OUT}/{RF}.xlsx")

## XGBoost

In [ ]:
# Parâmetros auxiliares (mude se desejar)
XGB_PARAMS = {
    "n_estimators": [100, 250, 500],
    "learning_rate": [0.05, 0.1],
    "max_depth": [3, 5, 7],
    "subsample": [0.8],
    "colsample_bytree": [0.8],
    "random_state": [RANDOM_STATE],
    "eval_metric": ["rmse"]
}

dict_cases[XGB] = get_model_cases(
    model=XGBRegressor(),
    params=XGB_PARAMS,
    pre_processing_cases=dict_cases[PRE],
    **CONFIG
)
dict_df_cases[XGB] = pd.DataFrame(data=dict_cases[XGB]).T

df_show_head(dict_df_cases[XGB])
dict_serialize(dict_cases[XGB], f"{PATH_OUT}/{XGB}.pkl")
generate_sheet(dict_cases[XGB], XGB, f"{PATH_OUT}/{XGB}.xlsx")